# <center> Fonctions projet chef-d'oeuvre </center>

---

## <center> Web scraping </center>

## Créer un nouveau csv

In [13]:
import csv

def creation_csv(nom): # il suffit d'indiquer le nom du csv (sans l'extension)
    header = ['Auteur(s)', 'Titre', 'Edition', 'Date', 'ISBN']
    with open(nom +'.csv', 'w') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(header)
    print("fichier créé avec le nom suivant :", nom)

In [92]:
creation_csv("test4")

fichier créé avec le nom suivant : test4


## Web-scraping (les Libraires.fr)

In [102]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm.notebook import tqdm_notebook


def web_scraping(nom_csv='test4', nb_livres=5000, nombre=205000, site = 'https://www.leslibraires.fr/livre/'):
    liste_totale = []
    for livre in tqdm_notebook(range(nb_livres)) : 
        erreur = 0
        
        try : 
            url = site + str(nombre)
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')

            liste = []

            # auteur 
            auteur = soup.find("h2")
            auteur = auteur.text.strip('\n')
            liste.append(auteur)

            # titre
            titre = soup.find("h1")
            titre = titre.text.strip('\n')
            liste.append(titre)

            # edition
            edition = soup.find("h3")
            edition = edition.text.strip('\n')
            liste.append(edition)

            # date
            date = soup.findAll("dd")
            # on récupère la date dans une variable temporaire
            var_temp = date[5].text
            # on split la date et on ne récupère que l'année
            split = var_temp.split("/")
            date = split[-1]
            # # on ajoute la date (format : int) à la liste
            # liste.append(int(date))
            liste.append(date)

            # ISBN
            # Récupérer l'isbn
            info = soup.findAll("dd")
            # on récupère l'isbn dans une variable temporaire
            isbn = info[2].text
            liste.append(isbn)

            liste_totale.append(liste)

            nombre += 1

        except :

            nombre += 1
            erreur += 1


    print("Nb d'erreurs :", erreur)

    # insérer les informations pour chaque livre (à partir de la liste totale)
    with open(nom_csv+'.csv', 'a', encoding='UTF8') as f:
            for x in tqdm_notebook(liste_totale) :
                writer = csv.writer(f)
                writer.writerow(x)

            # message de réussite
            print("Insertion des données réussies.")

In [103]:
web_scraping()

  0%|          | 0/5000 [00:00<?, ?it/s]

Nb d'erreurs : 0


  0%|          | 0/4625 [00:00<?, ?it/s]

Insertion des données réussies.


---

## <center> Formatage des données </center>

## Ouvrir un df

In [40]:
import pandas as pd

def open_df(file_name):
    df = pd.read_csv(file_name + '.csv')
    return df

In [105]:
df = open_df('test4')  
df.head()

,Auteur(s),Titre,Edition,Date,ISBN
0,Brookes John,Garden Design,Dorling Kinders,"26,9 x 23,9 x 3 cm",DORLING KINDERS
1,Cecil Saint-Laurent,Mutante Al,Flammarion,2006,9782082310352
2,mélanges offerts à Michel Sales,Lettre de l'esprit,Parole et silence,2005,9782845733282
3,De Jacques Pessis,Johnny Hallyday,Altinéa,2003,9782847630022
4,"De Dominique Balaÿ, Francis Balaÿ",Ile De Camargue - Il Etait Une Fois,"Illustrations de Jean Cubaud, Jean Roché",2007,9782848110592


## Récupérer des infos sur le df

In [106]:
def info_df (file_name):
    df = open_df(file_name)
    df.info()

In [107]:
info_df('test4')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4625 entries, 0 to 4624
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Auteur(s)  4625 non-null   object
 1   Titre      4625 non-null   object
 2   Edition    4625 non-null   object
 3   Date       4624 non-null   object
 4   ISBN       4625 non-null   object
dtypes: object(5)
memory usage: 180.8+ KB


## Supprimer doublons dans le csv

In [108]:
def delete_duplicates(file_name):
    # Suppression
    df.drop_duplicates(subset=None, inplace=True)
    # Sauvegarde = écrase le csv
    df.to_csv(file_name + ".csv", index=False)
    print("Les doublons ont bien été supprimés.")
    return df

## Formater la colonne 'Auteur'

In [109]:
def formater_auteur(file_name):
    # replace
    df['Auteur(s)'] = df['Auteur(s)'].str.replace("De ", "", regex=False)
#     # Sauvegarde du csv 
#     df.to_csv(file_name + ".csv", index=False)
    # # None 
    df['Auteur(s)'] = df['Auteur(s)'].replace(to_replace =["[none]", "[None]", "nones", "none", "Inconnu", "inconnu", "Xxx", " . ",
                                                          "nouvelles", "nouvelle", "roman", "romans", "poème", "poèmes","Poèmes", "Poème", "poésie", "poésies", "récit", 
                                                          "récits", "Livre", "Livres", "livre", "livres", "Volume 2", "et autres nouvelles",
                                                          "<<<<< ", "<<<<<", "Volume 1", "[nouvelles]", "livre du professeur", "l'intégrale",
                                                          "Pro", "Prof", "théâtre", "Théâtre", "A Preciser", "Po", "France", "- Roman", "Roman",
                                                          "3","2", "Anonyme", "anthologie", "Inconnu / Inconnu", "essai", "essais", "5e", "Tome 1"], 
                                value ="None")

    # Collectif
    df['Auteur(s)'] = df['Auteur(s)'].replace(to_replace =["De Collectif", "Collectifs", "collectif", " Collectif", "De Collectifs",
                                                          "Collectif d'auteurs", " Collectif d'auteurs", "Collectifs Jeunesse", "Collectifs Lois",
                                                          "Collectif,", "Collectifs Gallimard"],
                                value = "Collectif")
    # Sauvegarde du csv 
    df.to_csv(file_name + ".csv", index=False)
    print("Formatage de la colonne 'Auteur' : OK")
    return df

## Formatage de la colonne 'Edition'

In [110]:
def formater_edition(file_name):
    # Formatage des éditions 
    df['Edition'] = df['Edition'].str.replace("L'Harmattan", "Harmattan", regex=False)
    df['Edition'] = df['Edition'].str.replace("Presses universitaires de France", "PUF", regex=False)
    df['Edition'] = df['Edition'].str.replace("Editions Ellipses", "Ellipses", regex=False)
    df['Edition'] = df['Edition'].str.replace("Éditions du Rocher", "Rocher", regex=False)
    df['Edition'] = df['Edition'].str.replace("Editions du Rocher", "Rocher", regex=False)
    df['Edition'] = df['Edition'].str.replace("Editions ", "", regex=False)
    df['Edition'] = df['Edition'].str.replace("Éditions ", "", regex=False)

    # Sauvegarde du csv 
    df.to_csv(file_name + ".csv", index=False)
    print("Formatage de la colonne 'Edition' : OK")
    return df

---

## Traitement des ISBN

In [113]:
# Suppression des ISBN non chiffrés
from tqdm.notebook import tqdm_notebook

def traitement_isbn(nom_csv):
    liste=[]
    index = 0
    for x in tqdm_notebook(df['ISBN']):
        try :
            test = int(x)
            index += 1
        except :
            df.drop([index], axis=0, inplace=True)
            index +=1 

    # Sauvegarde
    df.to_csv(nom_csv+".csv", index=False)
    
    return df


In [114]:
# Trouver des ISBN aberrants :

def min_max_isbn():
    print("Min : ", df['ISBN'].min())
    print("Max : ", df['ISBN'].max())

In [115]:
# Supprimer les ISBN aberrants


def delete_bad_isbn(csv_name):
    index = -1
    liste = []

    for x in df['ISBN']:
        index += 1
        if x <= 9780000000000 :
            liste.append(index)
        
    # message    
    print("Nb d'erreurs d'ISBN :", len(liste))
    # delete
    df.drop(liste, axis=0, inplace=True)
    # save
    df.to_csv(csv_name + ".csv", index=False)
    
    return df.info()

## Traitement sur les dates

In [116]:
# Formatage
def formater_date(csv_name):
    # formatage des dates
    df['Date'] = df['Date'].replace(to_replace =["©","© ","Imp ","imp","imp ","impr ","Impr ","DL ","dl ","ilmpr","ilmpr ",
        "cop","Cop","ca","(c)","Paris PUF ","Paris PUF","Paris Gallimard ","Paris Gallimard","Paris Flammarion ",
        "Paris Flammarion","Paris Les Belles Lettres ","Paris Les Belles Lettres","Paris L'Harmattan ","Paris L'Harmattan",
        "Paris Hachette ","Paris Hachette","Paris Colin","Paris Colin " 
        ],
        value = "")
    # suppression des dates non numériques
    liste=[]
    index = 0
    for x in tqdm_notebook(df['Date']):
        try :
            test = int(x)
            index += 1
        except :
            df.drop([index], axis=0, inplace=True)
            index +=1 

    # Sauvegarde du csv 
    df.to_csv(csv_name +".csv", index=False)
    
    return df

In [117]:
# Statistiques sur les dates

def stats_date():
    print("Moyenne :", round(df.Date.mean(), 2))
    print("Min :", df.Date.min())
    print("Max :", df.Date.max())

In [118]:
# supression des dates aberrantes 

def delete_bad_dates(csv_name):
    nombre = 0
    index = -1
    for x in df['Date']:
        index += 1
        if x < 1700 :
            df.drop([index], axis=0, inplace=True)
            nombre += 1
    # Save
    df.to_csv(csv_name+".csv", index=False)

## Concaténation

In [119]:
def concat_csv(csv_1, csv_2, nom_csv):
    df = pd.concat(
    map(pd.read_csv, [csv_1+'.csv',csv_2+'.csv']), ignore_index=True)
    # Sauvegarde
    df.to_csv(nom_csv+".csv", index=False)

---

# <center> Traitement du csv </center>

In [169]:
nom_csv = '43'

In [170]:
df = open_df(nom_csv)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129345 entries, 0 to 129344
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Auteur(s)  128768 non-null  object
 1   Titre      129345 non-null  object
 2   Edition    129167 non-null  object
 3   Date       129345 non-null  int64 
 4   ISBN       129345 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 4.9+ MB


## Formater Auteur/Edition

In [166]:
def traitement_csv(file_name):
    df = open_df(file_name)
    df = delete_duplicates(file_name)
    df = formater_auteur(file_name)
    df = formater_edition(file_name)
    return df

In [167]:
traitement_csv(nom_csv)

Les doublons ont bien été supprimés.
Formatage de la colonne 'Auteur' : OK
Formatage de la colonne 'Edition' : OK


,Auteur(s),Titre,Edition,Date,ISBN
0,Reynal Sorel,Orphee Et L Orphisme,Que sais-je ?,1998,9782130472100
1,Gilbert Delahaye,Martine à l'cinse - Martine à la ferme en Ch'ti,Illustrations de Marcel Marlier,2006,9782203104730
2,Traduit par Daniel Garance,100 Techniques De Pros Pour Le Cinema Amateur,Eyrolles,2006,9782212119060
3,Satprem,La légende de l'avenir,Robert Laffont,2000,9782221091890
4,"L'Imitation (fragments, Épitomé) - Première le...",Opuscules rhétoriques / Denys d'Halicarnasse.\...,Édité par Germaine Aujac,1992,9782251004310
...,...,...,...,...,...
12200,None,6 - Intégrale Chick Bill - Tome 6 - Intégrale ...,Le Lombard,2001,9782803616220
12222,None,Des musées et des femmes et autres nouvelles,Traduit par Georges Magnane,1975,9782070291700
12241,None,Abécédaire de l'Apocalypse [Board book] Parnov...,Traduit par Bernard Kreise,2008,9782355860010
12242,None,Un pont sur la Loire,Table Ronde,2006,9782710328490


## Formater ISBN/Dates

In [168]:
# nom_csv = 'test4'
traitement_isbn(nom_csv)
df = open_df(nom_csv)

delete_bad_isbn(nom_csv)
df = open_df(nom_csv)

formater_date(nom_csv)
df = open_df(nom_csv)

delete_bad_dates(nom_csv)
df = open_df(nom_csv)
df.info()

  0%|          | 0/6527 [00:00<?, ?it/s]

Nb d'erreurs d'ISBN : 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6527 entries, 0 to 6526
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Auteur(s)  6511 non-null   object
 1   Titre      6527 non-null   object
 2   Edition    6527 non-null   object
 3   Date       6527 non-null   int64 
 4   ISBN       6527 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 255.1+ KB


  0%|          | 0/6527 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6527 entries, 0 to 6526
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Auteur(s)  6511 non-null   object
 1   Titre      6527 non-null   object
 2   Edition    6527 non-null   object
 3   Date       6527 non-null   int64 
 4   ISBN       6527 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 255.1+ KB


## Concaténation

In [163]:
concat_csv("test3", nom_csv, "test5")